<a href="https://colab.research.google.com/github/Nhok95/MDS_MachineLearning/blob/master/lab01/ml01_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML lab 01 - Data preprocessing

In [1]:
%load_ext autoreload

In [ ]:
# Uncomment to upgrade packages
#!pip3 install pandas --user --upgrade --quiet
#!pip3 install numpy --user --upgrade --quiet
#!pip3 install scipy --user --upgrade --quiet
#!pip3 install statsmodels --user --upgrade --quiet
#!pip3 install seaborn --user --upgrade --quiet

In [2]:
# Uncomment to upgrade packages (laptop)
!pip install pandas --upgrade --quiet
!pip install numpy --upgrade --quiet
!pip install scipy --upgrade --quiet
!pip install statsmodels --upgrade --quiet
!pip install seaborn --upgrade --quiet
!pip install sklearn --upgrade --quiet

     |████████████████████████████████| 38.1 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 9.8 MB 5.5 MB/s 


In [3]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.precision', 3)

In [4]:
# extra imports
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox

from google.colab import drive
import os 

In [5]:
pd.__version__

'1.3.5'

# SECTION 0: Mount Drive 
In this section I want to recover my files from the corresponding folder.

In [6]:
try:
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount, force_remount=True)

  # Switch to the directory on the Google Drive that you want to use
  drive_root = mount + '/MyDrive/Q2/ML/Lab01'

  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root



We're running Colab
Colab: mounting Google drive on  /content/drive
Mounted at /content/drive

Colab: making sure  /content/drive/MyDrive/Q2/ML/Lab01  exists.

Colab: Changing directory to  /content/drive/MyDrive/Q2/ML/Lab01
/content/drive/MyDrive/Q2/ML/Lab01


In [7]:
!pwd

/content/drive/MyDrive/Q2/ML/Lab01


## SECTION 1: READING THE FILE CREDSCO.TXT (loan data: credit scoring)

 Reading properly a data set is non-trivial because you need to know
 its data format: decimal separator, column separator, is there a
 header? how are strings quoted? how (if any) are missing values
 coded? should character vectors be converted to factors? should
 white spaces be stripped?, ...)

 It is a good idea to consult  [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)  and play with useful control parameters.
 
 after opening the file `credsco.csv` and inspecting it, we decide the following settings:

In [8]:
#header=0  => Denotes the first line of data rather than the first line of the file.
#sep=',' or delimiter=',' => Delimiter to use
Credit = read_csv("credsco.csv", header=0, delimiter=',')

#shape => Return a tuple representing the dimensionality of the DataFrame.
Credit.shape

(4455, 14)

 Basic questions:

* Which is the target variable? where is it? how many different values? is it a classification problem or a regression problem?

* *answers:* the target variable is located in column 1 and is called 'Assessment'; it has two possible values (therfore it is a classification problem)

What are the other variables?

In [9]:
#columns => The column labels of the DataFrame.
Credit.columns

Index(['Assessment', 'YearsInJob', 'Housing', 'Deadline', 'Age',
       'MaritalStatus', 'Records', 'TypeOfJob', 'Expenses', 'Income',
       'Capital', 'ChargesOnCapital', 'AmountRequested', 'MarketPrice'],
      dtype='object')

In [10]:
#First 4 rows
Credit[:4]

,Assessment,YearsInJob,Housing,Deadline,Age,MaritalStatus,Records,TypeOfJob,Expenses,Income,Capital,ChargesOnCapital,AmountRequested,MarketPrice
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325


inspect predictive variables 4, 5, 6 and 7 for the first example

In [ ]:
Credit.iloc[0,4:8]

Alternatively you can use the variable names

In [ ]:
Credit.loc[0,'Age':'TypeOfJob']

***
## SECTION 2: BASIC INSPECTION OF THE DATASET

Perform a basic inspection of the dataset. Have a look at the minimum and maximum values for each variable; find possible errors and abnormal values (outliers); find possible missing values; decide which variables are continuous and which are categorical; if there are mixed types, we have three options: recode continuous to categorical, recode categorical to continuous or leave them as they are. In the latter case, either the method accepts both kinds of information, or it does not, in which case python will convert the categorical ones to continuous using a dummy code.

In [ ]:
Credit.describe()

 `Assessment`,`Housing`,`MaritalStatus`,`Records`,`TypeOfJob` are categorical and need to be treated properly

 In particular, `Assessment` is the target variable; we need to identify correct values

 `Capital`, `ChargesOnCapital` and `Income` present abnormally high maximums (99999999)

 There are also suspicious zeros, in both types of variables, which we identify with missing values

In [ ]:
Credit['Assessment'].value_counts()

## SECTION 3: DEALING WITH MISSING VALUES

Sometimes we need to take a decision on a sensible treatment for the missing values and apply it; it is wise to write down the possible consequences of this decision and the alternatives that could be considered in case the final results are not satisfactory

the easiest way is of course to eliminate the involved rows or
columns; this can be done partially. For example, we could decide to
eliminate the variables with the highest proportion of missing values.

Deleting instances and/or variables containing missing values results
in loss of relevant data and is also frustrating because of the effort
in collecting the sacrificed information.

<div class="alert alert-block alert-warning">
CAREFUL! python does not know magically which entries are missing values: they have to be explicitly declared as NA's
</div>

therefore this code is not useful:

In [ ]:
Credit_complete = Credit.dropna()
Credit_complete.shape

 the previous code does nothing! (but it seems it does)

 In the present case we have decided to perform a step-by-step treatment, separate for the categorical and continuous information

 We first decide to remove those rows with with missing values in the categorical variables (there are few)

In [ ]:
(Credit.Assessment==0).value_counts()
(Credit.Housing==0).value_counts()
(Credit.MaritalStatus==0).value_counts()
(Credit.TypeOfJob==0).value_counts()

In [ ]:
Credit=  Credit[(Credit.Assessment!=0) & (Credit.Housing!=0)
                &(Credit.MaritalStatus!=0)&(Credit.TypeOfJob!=0)]
Credit.shape

 Process rows with missing values in the continuous variables (code 99999999)

 look at that:

In [ ]:
Credit.Income.hist(figsize=(8,8));

In [ ]:
Credit.Income[Credit.Income!=99999999].hist(figsize=(8,8));

In [ ]:
Credit.Income[(Credit.Income!=99999999)&
              (Credit.Income!=0)].hist(bins=15,figsize=(8,8));

 these are then clearly incorrect

In [ ]:
(Credit.Income==99999999).value_counts()
(Credit.Income==0).value_counts()
(Credit.Capital==99999999).value_counts()
(Credit.ChargesOnCapital==99999999).value_counts()

In [ ]:
(Credit.Income==99999999).value_counts()

 what do we do with this one? let's assume it is correct

In [ ]:
(Credit.YearsInJob==0).value_counts()

 Continuous variables have too many missing values, we can not eliminate them just like that: we must devise a treatment for these missing values

 first we mark them to 'NA', including those from no 'Income'

In [ ]:
Credit.Income[(Credit.Income == 99999999) | (Credit.Income == 0)] = np.nan
Credit.Capital[Credit.Capital == 99999999]  = np.nan
Credit.ChargesOnCapital[Credit.ChargesOnCapital == 99999999]  = np.nan

see the difference?

In [ ]:
Credit.Income.describe()

 The word 'imputation' refers to assigning a value to every missing value. Here we perform imputation by a method known as 1NN: for every individual with a missing 'Income', we look for the most similar individual (according to the remaining variables) and then copy its 'Income' value.
 
 As we can not have missing values in any column for computing the 1KNN we will make a classifier dropping the columns with missing values and training a classifier for each column
 

In [ ]:
Credit.shape

In [ ]:
subset_of_credit = Credit.drop(columns=['Income', 'Capital', 'ChargesOnCapital'])
subset_of_credit.shape

rows_not_missing = Credit.Income.notna() & Credit.Capital.notna() & Credit.ChargesOnCapital.notna()

credit_without_missings = subset_of_credit[rows_not_missing]
credit_without_missings.shape

credit_where_income_has_missing_values = subset_of_credit[Credit.Income.isna()]
credit_where_income_has_missing_values.shape

 Neither of credit_without_missings, credit_where_income_has_missing_values can contain NAs

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(credit_without_missings, Credit.Income[rows_not_missing])
knn_inc = knn.predict(credit_where_income_has_missing_values);

 Imputation of 'Capital'

In [ ]:
capital_missing_values = subset_of_credit[Credit.Capital.isna()]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(credit_without_missings, Credit.Capital[rows_not_missing])
knn_cap = knn.predict(capital_missing_values)

Imputation of 'ChargesOnCapital'

In [ ]:
charges_missing_values = subset_of_credit[Credit.ChargesOnCapital.isna()]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(credit_without_missings, Credit.ChargesOnCapital[rows_not_missing])
knn_cop = knn.predict(charges_missing_values);

In [ ]:
Credit.Income[Credit.Income.isna()] = knn_inc
Credit.Capital[Credit.Capital.isna()] = knn_cap
Credit.ChargesOnCapital[Credit.ChargesOnCapital.isna()] = knn_cop
Credit.ChargesOnCapital[Credit.Capital==0] = 0

There are computanionally cheaper methods for missing value imputations such as replacing with mean, median or mode. All these can be computed using pandas `replace` and `fillna` functions, you can find more info [here](https://pandas.pydata.org/pandas-docs/stable/missing_data.html)

The Scikit-learn library has also a method for the most simple missing value imputation [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html)

inspect again the result, especially the new statistics

In [ ]:
Credit.shape
Credit.describe()

## SECTION 4: FINDING OUTLIERS

In real data you are usually going to find outliers. It can be because the data is just like that or because there are wrong values stored. 

It is important to identify them so you can remove them, impute them, or just acknowledge their existence and take into account in your analysis. 

Some machine learning models are very sensitive to outliers. 

There are a lot of ways to define an outlier. Here we are going to talk about a basic and a complex one. 

In [ ]:
fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))
Credit.boxplot(column='MarketPrice',ax=axes[0]);
Credit.hist(column='MarketPrice', ax=axes[1]);

### IQR 

One easy criteria to decide which values are outliers is to use the distance between quartiles. Specifically, you define the outliers as: 
* The values smaller than Q1 - 1.5*IQR
* The values bigger than Q3 + 1.5*IQR

This is the criteria used in the boxplots to marc the outliers. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Boxplot_vs_PDF.svg/1024px-Boxplot_vs_PDF.svg.png" alt="nice_image_from_wikipedia" width="450"/>

If we used this criteria to remove the outliers of our variable we would get the next distribution.

In [ ]:
Q1 = Credit['MarketPrice'].quantile(0.25)
Q3 = Credit['MarketPrice'].quantile(0.75)
IQR = Q3 - Q1

Q1, Q3, IQR

In [ ]:
small_outliers = Credit['MarketPrice'] < (Q1 - 1.5 * IQR)
big_outliers = Credit['MarketPrice'] > (Q3 + 1.5 * IQR)

sum(small_outliers), sum(big_outliers)

In [ ]:
Credit['MarketPrice'][small_outliers | big_outliers].head()

In [ ]:
fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))
Credit[~(small_outliers | big_outliers)].boxplot(column='MarketPrice',ax=axes[0]);
Credit[~(small_outliers | big_outliers)].hist(column='MarketPrice', ax=axes[1]);

### Model based
Sometimes the classic approaches will not be enough to detect outliers. The data can take many forms, like images, text or just pairs of variables that would not make sense to take independenly for any reason. 

In these cases we will want to use more complex approaches. Here we are going to talk about [Local Outlier Factor](https://dl.acm.org/doi/abs/10.1145/342009.335388), but there are a lot more. 

Local outlier factor uses the local density of a point to decide whether it is an otlier or not. The local density is calculated by using its k nearest neighbors. 

<div class="alert alert-block alert-warning">
If you use a model-based approach you will need to tune its hyperparameters. 
</div>

In [ ]:
local_outlier_factor = LocalOutlierFactor(n_neighbors=20)
result = local_outlier_factor.fit_predict(Credit['MarketPrice'].values.reshape(-1, 1))

outliers = result == -1 
no_outliers = result == 1

In [ ]:
Counter(result)

In [ ]:
fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))
Credit[no_outliers].boxplot(column='MarketPrice',ax=axes[0]);
Credit[no_outliers].hist(column='MarketPrice', ax=axes[1]);

***
## SECTION 5: TREATMENT OF MIXED DATA TYPES


 In this case we have decided to keep the original type and leave the decision for later, depending on the specific analysis

 we explicitly declare categorical variables as such 

In [ ]:
Credit.dtypes

In [ ]:
# There is a categorical datatype in pandas, but for most things this will do
Credit.Assessment = Credit.Assessment.astype('object')
Credit.Housing = Credit.Housing.astype('object')
Credit.MaritalStatus = Credit.MaritalStatus.astype('object')
Credit.Records = Credit.Records.astype('object')
Credit.TypeOfJob = Credit.TypeOfJob.astype('object')

Credit.Assessment.unique()
Credit.Housing .unique()
Credit.MaritalStatus.unique()
Credit.Records.unique()
Credit.TypeOfJob.unique()


not very nice, right? let's recode

In [ ]:
Credit.Assessment.replace([1, 2], 
                          ["positive","negative"], 
                          inplace=True)
Credit.Housing.replace([1,2,3,4,5,6], 
                       ["rent","owner","private","ignore","parents","other"], 
                       inplace=True)
Credit.MaritalStatus.replace([1,2,3,4,5], 
                             ["single","married","widower","split","divorced"], 
                             inplace=True)
Credit.Records.replace([1, 2], 
                       ["no","yes"], inplace=True)
Credit.TypeOfJob.replace([1,2,3,4], 
                         ["indefinite","temporal","self-employed","other"], 
                         inplace=True)


<div class="alert alert-block alert-warning">
If you want to treat a categorical variable as numerical you should never replace the categories by numbers. 
If you do so you are adding an order relationship that did not exist previously in the data, adding noise to your analysis. 
</div>

***
## SECTION 6: DERIVATION OF NEW VARIABLES: FEATURE EXTRACTION

 We decide whether it can be sensible to derive new variables; we extract two new continuous and one new categorical variable (for the sake of illustration):

 Financing ratio (continuous)


In [ ]:
Credit['FinancingRatio'] = 100*Credit.AmountRequested/Credit.MarketPrice
Credit.FinancingRatio.hist(figsize=(8,8));

 Saving capacity (continuous)

In [ ]:
Credit['SavingCapacity'] = (Credit.Income- Credit.Expenses-(Credit.ChargesOnCapital/100))\
                            /(Credit.AmountRequested/Credit.Deadline)
Credit.SavingCapacity.hist(bins=16,figsize=(8,8));

 Amount Requested greater than the median by people younger than 1.25 times the mean (categorical):

In [ ]:
Credit['Dubious'] = ['No']*Credit.shape[0]
Credit.Dubious[(Credit.AmountRequested > Credit.AmountRequested.median(skipna=True)) &
               (Credit.Age < 1.25*Credit.Age.mean(skipna=True))] = "Yes"
pd.crosstab(Credit.Dubious, Credit.Assessment)

***
## SECTION 7: WHAT WE HAVE DONE SO FAR


 Create a new dataframe that gathers everything and inspect it again

In [ ]:
Credit_new =Credit.copy()
                   
Credit_new.describe(include='all')
Credit_new.shape

***
## SECTION 8: GAUSSIANITY AND TRANSFORMATIONS


 Perform a graphical summary of some of the variables (both categorical and continuous), using the boxplot() and hist() procedures

 For continuous data:
 histograms and boxplots

In [ ]:
Credit_new.Income.hist(bins=20,figsize=(8,8), color='green');

In [ ]:
Credit_new.Capital.hist(bins=20,figsize=(8,8), color='red');

In [ ]:
pd.options.mode.use_inf_as_na = True
Credit_new.Capital.apply(np.log10).hist(bins=20,figsize=(8,8), color='red');

In [ ]:
Credit_new.boxplot(column='Deadline',figsize=(8,8));

In [ ]:
Credit_new.boxplot(column='Age',figsize=(8,8));

In [ ]:
Credit_new.loc[:,"Expenses":"SavingCapacity"].boxplot(figsize=(14,8));

In [ ]:
Credit_new.loc[:,"Expenses":"SavingCapacity"].boxplot(figsize=(14,8),showfliers=False); # much better, but would be nicer one by one

 the previous plots suggest to take logs on some variables: Capital and ChargesOnCapital (we'll do it later)

 For categorical data:
 Frequency tables, Contingency tables, Bar charts, Pie charts

 should we treat Age as categorical? probably not

### Transform numerical variables into categorical

In [ ]:
Credit_new.Age.unique()

In [ ]:
Credit_new.Age.min()
Credit_new.Age.max()

In [ ]:
bins = pd.IntervalIndex.from_tuples([(0, 1), (2, 3), (4, 5)])
bins

In [ ]:
pd.interval_range(start=30, end=90,freq=10)

In [ ]:
pd.cut(Credit_new.Age,
       bins=pd.interval_range(start=30, end=90,freq=10))  
# WARNING! we are generating NAs               

In [ ]:
Age_cut = pd.cut(Credit_new.Age,
                 bins=pd.interval_range(start=15, end=75,freq=10)) ;

In [ ]:
Credit_new['Age_cat'] = Age_cut.astype('str')

In [ ]:
Credit_new.Age_cat.value_counts().sort_index()

In [ ]:
Credit_new.Age_cat.value_counts().sort_index().plot.bar(figsize=(8,8));

In [ ]:
Credit_new.Age_cat.value_counts().sort_index().plot.pie(figsize=(8,8));

 incidentally, this is how we could generate another new variable based on Age:

In [ ]:
Credit_new['Age2_cat'] = Credit_new.Age.apply(lambda x : 
                                              'under55' if x < 55 else 'over55')

In [ ]:
TypeOfJob_Age= pd.crosstab(Credit_new.TypeOfJob, Credit_new.Age_cat)
TypeOfJob_Age

In [ ]:
sn.heatmap(TypeOfJob_Age, annot=True, fmt="d", cmap='coolwarm')

In [ ]:
TypeOfJob_Age.sum(axis=0) # row sums
TypeOfJob_Age.sum(axis=1) # column sums

In [ ]:
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Age_cat,
            normalize=True,
            margins=True) # relative frequencies

In [ ]:
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Age_cat,
            normalize=True,margins=True).round(decimals=3)
            # idem, rounded to 3 digits

In [ ]:
(pd.crosstab(Credit_new.TypeOfJob,
             Credit_new.Age_cat,
             normalize=True,
             margins=True)*100).round(decimals=3) 
             # total percentages

In [ ]:
pd.crosstab(Credit_new.TypeOfJob,
            Credit_new.Age_cat,
            normalize="index").round(decimals=3) 
            # table of relative frequencies (column-wise)
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Age_cat,
            normalize="columns").round(decimals=3) 
            # table of relative frequencies (row-wise)

In [ ]:
# basic stacked bar chart
TypeOfJob_Age.T.plot.bar(stacked=True, figsize=(8,8));  

In [ ]:
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Age_cat,
            normalize="columns").T.plot.bar(stacked=True, figsize=(8,8));

In [ ]:
# grouped bar chart
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Age_cat,
            normalize="columns").T.plot.bar(figsize=(8,8));


###  One hot encoding (dummy encoding)

Sometimes you will want to treat some (of all) your categorical variables as numerical. This can happen if, for example you want to train a model that does not accept categorical variables. 
There are different methods to codify the categories. One of the most simple and comonly used is one hot encoding.  

In [ ]:
Credit['Records'].head()

In [ ]:
pd.get_dummies(Credit['Records'], prefix='Records').head()

### Compare variables 

In [ ]:
Credit_new.plot.scatter(y='Capital',
                        x='AmountRequested', 
                        figsize=(8,8),
                        title='Amount req. vs. Capital');

In [ ]:
Credit_new['Capital_log10'] = (Credit_new.Capital+1).apply(np.log10)
Credit_new['AmountRequested_log10'] = (Credit_new.AmountRequested).apply(np.log10)
Credit_new.plot.scatter(y='Capital_log10',
                        x='AmountRequested_log10', 
                        figsize=(8,8),
                        title='log Amount req. vs. log Capital (better)');

 adding a center (dashed) and a regression line (red)

In [ ]:

model = GLM.from_formula('Capital_log10 ~ AmountRequested_log10', Credit_new)
result = model.fit()

Credit_new.plot.scatter(y='Capital_log10',
                        x='AmountRequested_log10', 
                        figsize=(8,8))
plt.title('Amount req. vs. Capital (better)');
plt.plot([Credit_new.AmountRequested_log10.mean()]*2,
         [Credit_new.Capital_log10.min(),
          Credit_new.Capital_log10.max()],
         'k:');
plt.plot([Credit_new.AmountRequested_log10.min(),
          Credit_new.AmountRequested_log10.max()],
         [Credit_new.Capital_log10.mean()]*2,
         'k:');
plt.plot(np.linspace(Credit_new.AmountRequested_log10.min(), 
                     Credit_new.AmountRequested_log10.max(),
                     num=30),
         result.params.Intercept+
         result.params.AmountRequested_log10*
         np.linspace(Credit_new.AmountRequested_log10.min(),                                                             
                     Credit_new.AmountRequested_log10.max(),num=30),'r');


 (note that log10(x+1)=0 for x=0, so our transformation keeps the zeros)
     
 On the other hand, these same zeros spoil the regression: perhaps it would be more sensible to do the regression without them

In [ ]:
Credit_new.boxplot(column='AmountRequested',
                   by='TypeOfJob',
                   figsize=(12,8));

In [ ]:
Credit_new.boxplot(column='Age',
                   by='Assessment',
                   figsize=(12,8));

In [ ]:
pd.crosstab(Credit_new.TypeOfJob, 
            Credit_new.Assessment,
            normalize="columns").T.plot.bar(stacked=True, 
                                            figsize=(8,8));

In [ ]:
scatter_matrix(Credit_new.loc[:,['AmountRequested','Capital','Age']], 
               alpha=0.2, figsize=(12, 12), 
               diagonal='kde', marker='.');

### Gaussianity

In [ ]:
sigma=  Credit_new.Expenses.std()
mu=  Credit_new.Expenses.mean()
fig, ax = plt.subplots(figsize=(8,8))
count, bins, ignored = plt.hist(Credit_new.Expenses, 15, density=True)
dbins = np.linspace(bins[0], bins[-1])
plt.title("Expenses")
ax.set_xlabel("Expenses")
plt.plot(dbins, 1/(sigma * np.sqrt(2 * np.pi)) *
                np.exp( - (dbins - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='r');

In [ ]:
fig = plt.figure(figsize=(12,12))
for i in range(8):
    ax = fig.add_subplot(3, 3, i+1)
    Credit_new[Credit.columns[i+8]].plot.kde()
    plt.title(Credit.columns[i+8])
fig.tight_layout();

In [ ]:
Credit_new.loc[:,'Expenses':'SavingCapacity'].plot.kde(subplots=True,
                                                       layout=(4,3),
                                                       sharex=False, 
                                                       figsize=(12,12));

 do any of the continuous variables "look" Gaussian? 
 features to look for in comparing to a Gaussian: outliers, asymmetries, long tails

## SECTION:9 NORMALIZATION

If you try to train a model with varaibles of ranges too far away it will be a disaster most of the times. Becasue the model might only "see" the bigger variable. 

To avoid this issue you usualy normalize or standarize your data. This way you force all your variables to have the same range. 
There are models that are very sensitive to this and might even fail to converge if you don't normalize your data. 

The most comon transformations for normalizing the data are: 
* Standarization: $\frac{X - \mu}{\sigma}$ will trasnform your data so it has mean 0 and std 1.
* Min-max scaling: $\frac{X - X_{min}}{X_{max} - X_{min}}$ will send your data to the range [0,1]
* Boxcox transformation: $\frac{X^\lambda - 1}{ \lambda}$ if $\lambda \neq 0$ or $ln(X)$ if $\lambda = 0$ transforms the data to try to fit a normal distribution. 


In [ ]:
Credit_new[['Age', 'MarketPrice']].describe()

In [ ]:
min_max_scaler =  preprocessing.MinMaxScaler()
Credit_new[['Age_min_max', 'MarketPrice_min_max']] = min_max_scaler.fit_transform(Credit_new[['Age', 'MarketPrice']])
std_scaler = preprocessing.StandardScaler()
Credit_new[['Age_std','MarketPrice_std']]= preprocessing.scale(Credit_new[['Age', 'MarketPrice']])

x, _= boxcox(Credit_new['Age'])
Credit_new['Age_bx'] = x

x, _= boxcox(Credit_new['MarketPrice'])
Credit_new['MarketPrice_bx'] = x

In [ ]:
Credit_new[['Age_min_max', 'MarketPrice_min_max','Age_std', 'MarketPrice_std', 'Age_bx','MarketPrice_bx']].describe()

In [ ]:
Credit_new[['Age','Age_min_max', 'Age_std', 'Age_bx']].hist(figsize=(14,4), layout=(1,4));

In [ ]:
Credit_new[['MarketPrice','MarketPrice_min_max', 'MarketPrice_std', 'MarketPrice_bx']].hist(figsize=(14,4), layout=(1,4));

extra plot boxcox transformation: 

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1, 2, 1)
Credit_new.Capital.plot.hist(title='Look at that ...')

# the boxcox function transforms the data using 
# the power transformation (x**lambda -1)/ lambda
# the function takes care of finding the optimal lambda
x, _= boxcox(Credit_new.Capital+1)

Credit_new['Capital_BC'] = x
ax = fig.add_subplot(1, 2, 2)
Credit_new.Capital_BC.plot.hist(title='Look at that now ...');

## SECTION 10: ENDING THE PREPROCESSING

  
 Shuffle the data (to avoid possible ordering biases)

In [ ]:
np.random.seed(144)
Credit_new = Credit_new.sample(frac=1).reset_index(drop=True)

 
 Save the preprocessed data into a file for future use
<div class="alert alert-block alert-warning">
Pandas allows to save the data in a lot of different formats as you can see [here](http://pandas-docs.github.io/pandas-docs-travis/io.html) among others CSV, pickle, HDF5, JSON, Excel as well as other data storages like SQL databases, Google Big Query, parquet or feather.

The simplest way is to save the data as a csv with `to_csv` or as a pickle file (native python store format) with `to_pickle`, this last one allows also to compress the data.
</div>


In [ ]:
Credit_new.to_pickle('Credsco-processed.pkl.bz2',compression='bz2')

In [ ]:
Credit_load =pd.read_pickle('Credsco-processed.pkl.bz2',compression='bz2')
Credit_load.head()